In [3]:
pip install fastapi 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
from io import StringIO
from fastapi import FastAPI, Query
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware

##############################
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}
###############################

# Inicializar Faker y FastAPI
fake = Faker('es_ES')
app = FastAPI()

# Configurar CORS para permitir peticiones desde cualquier origen (opcional)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Resto de tu código de generación de datos

nombres_hombres = pd.read_excel(r'Nombres_Edades.xlsx', sheet_name='Hombres')
nombres_mujeres = pd.read_excel(r'Nombres_Edades.xlsx', sheet_name='Mujeres')
distribucion_edades = pd.read_excel(r'Nombres_Edades.xlsx', sheet_name='Distribucion_Edades')
distribucion_sexo = pd.read_excel(r'Nombres_Edades.xlsx', sheet_name='Distribucion_Sexo')
apellidos = pd.read_excel(r'Apellidos.xlsx', sheet_name='Apellidos')

# Procesamiento de apellidos y nombres
apellidos = apellidos.drop(columns='Unnamed: 0')
primer_apellido = apellidos['Primer_Apellido'].dropna()
segundo_apellido = apellidos['Segundo_Apellido'].dropna()
prob_primer_apellido = apellidos['Porcentaje_Primer_Apellido'].dropna()
prob_segundo_apellido = apellidos['Porcentaje_Segundo_Apellido'].dropna()

# Funciones de generación de datos

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

def generar_sueldo(grupo, grupos_caracteristicas):
    return round(np.random.uniform(*grupos_caracteristicas[grupo]['caracteristicas']['rango_sueldo']), 2)

def calcular_dinero_banco(sueldo):
    return round(np.random.uniform(sueldo * 3, sueldo * 12), 2)

def generar_edad(rango_edad):
    return random.randint(*rango_edad)

def generar_hijos(grupo, grupos_caracteristicas):
    min_hijos, max_hijos = grupos_caracteristicas[grupo]['caracteristicas']['rango_hijos']
    return random.randint(min_hijos, max_hijos)

def generar_sexo(edad, distribucion_sexo, rangos_edad):
    rango = next((r for r in rangos_edad if int(r.split(' - ')[0]) <= edad <= int(r.split(' - ')[1])), "85+")
    dist = distribucion_sexo[distribucion_sexo['Rango_Edades'] == rango].iloc[0]
    return np.random.choice(['H', 'M'], p=[dist['Porcentaje_Hombres'], dist['Porcentaje_Mujeres']])

def generar_nombre(edad, sexo, nombres_hombres, nombres_mujeres, rangos_edad):
    rango = next((r for r in rangos_edad if int(r.split(' - ')[0]) <= edad <= int(r.split(' - ')[1])), "85+")
    nombres = nombres_hombres if sexo == 'H' else nombres_mujeres
    nombres_filtrados = nombres[nombres['Rango_Edad'] == rango]
    return np.random.choice(
        nombres_filtrados[f'Nombre_{"Hombre" if sexo == "H" else "Mujer"}'].values,
        p=nombres_filtrados[f'Porcentaje_Nombre_{"Hombre" if sexo == "H" else "Mujer"}'].values
    )

def generar_apellidos(apellidos_df):
    primer_apellido_probs = apellidos_df['Porcentaje_Primer_Apellido'].fillna(0)
    primer_apellido_probs /= primer_apellido_probs.sum()
    segundo_apellido_probs = apellidos_df['Porcentaje_Segundo_Apellido'].fillna(0)
    segundo_apellido_probs /= segundo_apellido_probs.sum()
    
    primer_apellido = np.random.choice(apellidos_df['Primer_Apellido'], p=primer_apellido_probs)
    segundo_apellido = np.random.choice(apellidos_df['Segundo_Apellido'], p=segundo_apellido_probs)
    
    return f"{primer_apellido} {segundo_apellido}"

def generar_iban_kutxabank():
    entidad = "2095"
    oficina = f"{random.randint(0, 9999):04d}"
    dc = f"{random.randint(0, 99):02d}"
    cuenta = f"{random.randint(0, 9999999999):010d}"
    return f"ES95{entidad}{oficina}{dc}{cuenta}"

# Función para generar usuarios (simplificado)
def generar_usuario(grupo, caracteristicas, rango_edad, distribucion_sexo, nombres_hombres, nombres_mujeres, rangos_edad, apellidos):
    edad = generar_edad(rango_edad)
    sexo = generar_sexo(edad, distribucion_sexo, rangos_edad)
    sueldo = generar_sueldo(grupo, caracteristicas)
    
    usuario = {
        'grupo': grupo,
        'edad': edad,
        'sexo': sexo,
        'sueldo': sueldo,
        'dinero_banco': calcular_dinero_banco(sueldo),
        'hijos': generar_hijos(grupo, caracteristicas),
        'vivienda': np.random.choice(['Propiedad', 'Alquiler'], p=[caracteristicas['caracteristicas']['prob_casa'], 1 - caracteristicas['caracteristicas']['prob_casa']]),
        'coche': random.random() < caracteristicas['caracteristicas']['prob_coche'],
        'nombre': generar_nombre(edad, sexo, nombres_hombres, nombres_mujeres, rangos_edad),
        'apellidos': generar_apellidos(apellidos),
        'iban': generar_iban_kutxabank()
    }
    return usuario

# Función para generar las transacciones (modificada para incluir fechas y cantidad)
def generar_transacciones_vectorizado(cantidad, fecha_inicio, fecha_fin):
    # Lógica para generar las transacciones
    # Simulación de DataFrame con transacciones ficticias
    fechas = pd.date_range(start=fecha_inicio, end=fecha_fin)
    data = {
        "fecha": np.random.choice(fechas, cantidad),
        "tipo": np.random.choice(["Compra", "Venta"], cantidad),
        "importe": np.random.uniform(100, 1000, cantidad),
        "iban": [generar_iban_kutxabank() for _ in range(cantidad)]
    }
    return pd.DataFrame(data)

# Endpoint de FastAPI para que el usuario ingrese cantidad y fechas
@app.get("/generar_transacciones/")
def generar_transacciones(
    cantidad: int = Query(..., description="Cantidad de transacciones a generar"),
    fecha_inicio: str = Query(..., description="Fecha de inicio en formato YYYY-MM-DD"),
    fecha_fin: str = Query(..., description="Fecha de fin en formato YYYY-MM-DD")
):
    # Convertimos las fechas de entrada en objetos datetime
    fecha_inicio_dt = datetime.strptime(fecha_inicio, "%Y-%m-%d")
    fecha_fin_dt = datetime.strptime(fecha_fin, "%Y-%m-%d")

    # Generamos las transacciones
    transacciones = generar_transacciones_vectorizado(cantidad, fecha_inicio_dt, fecha_fin_dt)

    # Convertimos el DataFrame en un archivo CSV
    output = StringIO()
    transacciones.to_csv(output, index=False)
    output.seek(0)

    # Devolvemos el CSV como respuesta
    return StreamingResponse(output, media_type="text/csv", headers={"Content-Disposition": "attachment; filename=transacciones.csv"})

